# Data Wrangling
In this notebook we'll obtain all of our data, arrange it how we want, and store it in the proper directories.

In [ ]:
%run setup.ipynb
import copernicusmarine

tg_nc_dir = data_dir / 'tide_gauge_nc'

# Check if the directories exist, if not create them
for d in [data_dir, output_dir, tg_nc_dir]:
    if not d.exists():
        d.mkdir()

# make a climate_indices directory if it doesn't exist
climate_indices_dir = data_dir / 'climate_indices'
if not climate_indices_dir.exists():
    climate_indices_dir.mkdir()        

## Retrieve Tide Gauge Data

We are interested in getting tide gauge and alitmetry data for the Hawaiian Islands (and surrounds) for 1993 through 2022.
Let's first establish where the tide gauges are by looking at the tide gauge dataset. We'll retrieve tide gauge data from the UHSLC (University of Hawaii Sea Level Center) fast-delivery dataset {cite:t}`patrick_c_caldwell_sea_2015`. The fast-delivery data are released within 1-2 months of data collection and are subject only to basic quality control. 

We'll be retrieving the hourly data for our station group at from UHSLC, and saving this to our data directory so we don't have to download again.

In [ ]:
# hawaii stations are: 
stationdict = {
    'Hilo': '060',
    'Kawaihae': '552',
    'Kahului': '059',
    'Mokuoloe': '061',
    'Honolulu': '057',
    'Nawiliwili': '058',
    'Johnston Island': '052',
    'Midway Island': '050',
    'Kaumalapau': '548',
    'Barbers Point': '547',
    'French Frigate Shoals': '014',
}
stationdict.values()

station_group = 'Hawaiian Islands'

glue('station_group', station_group)

````{margin}
```{note}
What about research quality data (RQD)? 
RQD undergo thorough and time-consuming QC, and are usually released 1-2 years after data is received. 
```
````

In [ ]:
url = "https://uhslc.soest.hawaii.edu/data/netcdf/fast/hourly/" 
uhslc_ids = list(stationdict.values())

for uhslc_id in uhslc_ids:
    fname = f'h{uhslc_id}.nc' # h for hourly, d for daily

    path = os.path.join(data_dir, 'tide_gauge_nc',fname)

    if not os.path.exists(path):
        urlretrieve(os.path.join(url, fname), path) 
        print(f'Downloading {fname} from {url} to {path}')

Now we merge all the datasets. This can take a while.

```{caution}
In the following section I remove the trailing zero from the record-id of each tide gauge. This trailing zero is only on fast-delivery data products.
```

In [ ]:
# Load the data

import glob        
# Get a list of all .nc files in the data directory
# filenames should be in the format h*.nc with * being the station id in the stationdict
filenames = 'h' + pd.Series(stationdict.values()) + '.nc'
files = [glob.glob(str(data_dir / 'tide_gauge_nc' / file))[0] for file in filenames]


# Open the datasets
datasets = [xr.open_dataset(file) for file in files]

#merge in batches of 2 to avoid memory issues
batch_size = 2
merged_datasets = []

for i in range(0, len(datasets), batch_size):
    batch = datasets[i:i+batch_size]
    merged_batch = xr.merge(batch, compat='no_conflicts', join='outer')
    merged_datasets.append(merged_batch)

#merge the merged datasets
rsl = xr.merge(merged_datasets, compat='no_conflicts', join='outer')



In [ ]:
# rename record_id to station_id - this will align with NOAA and other data sources
rsl = rsl.rename({'record_id': 'station_id'})

In [ ]:
# convert byte strings to normal strings
rsl['station_name'] = xr.DataArray(
    np.array(rsl['station_name'].values, dtype='U50'),
    dims=('station_id',)
)
rsl['station_country'] = xr.DataArray(
    np.array(rsl['station_country'].values, dtype='U30'),
    dims=('station_id',)
)
rsl['ssc_id'] = xr.DataArray(
    np.array(rsl['ssc_id'].values, dtype='U30'),
    dims=('station_id',)
)

# remove the trailing zero from each station_id
rsl['station_id'] =(rsl['station_id']/10).astype(int)
rsl

In [ ]:
def get_MHHW_uhslc_datums(id, datumname, table=None):
    
    url = 'https://uhslc.soest.hawaii.edu/stations/TIDES_DATUMS/fd/LST/fd'+f'{int(id):03}'+'/datumTable_'+f'{int(id):03}'+'_m_GMT.csv'
    datumtable = pd.read_csv(url)
    datum = datumtable[datumtable['Name'] == datumname]['Value'].values[0]
    # ensure datum is a float
    datum = float(datum)
    if table:
        return datum, datumtable
    else:
        return datum

In [ ]:
# add MHHW to the dataset
rsl['MHHW'] = xr.DataArray([get_MHHW_uhslc_datums(id, 'MHHW') for id in rsl['uhslc_id'].values], dims='station_id', coords={'station_id': rsl['station_id']})

rsl['MHHW'].attrs['units'] = 'm'
rsl['MHHW'].attrs['long_name'] = 'Mean Higher High Water, rel. to station datum'

glue('datumname', 'MHHW')

# add MSL to the dataset
rsl['MSL'] = xr.DataArray([get_MHHW_uhslc_datums(id, 'MSL') for id in rsl['uhslc_id'].values], dims='station_id', coords={'station_id': rsl['station_id']})

rsl['MSL'].attrs['units'] = 'm'
rsl['MSL'].attrs['long_name'] = 'Mean Sea Level, rel. to station datum'

# add MSL to the dataset
rsl['MLLW'] = xr.DataArray([get_MHHW_uhslc_datums(id, 'MLLW') for id in rsl['uhslc_id'].values], dims='station_id', coords={'station_id': rsl['station_id']})

rsl['MLLW'].attrs['units'] = 'm'
rsl['MLLW'].attrs['long_name'] = 'Mean Low Low Water, rel. to station datum'



In [ ]:
#round the time to the nearest hour
rsl['time'] = rsl['time'].dt.round('h')

In [ ]:
#convert sea level variable to m
rsl['sea_level'] = rsl['sea_level'] / 1000  # convert from mm to m
rsl['sea_level'].attrs['units'] = 'm'

In [ ]:
#ensure rsl is netcdf4 format
rsl.to_netcdf(data_dir / 'rsl_hawaii_uhslc.nc',engine='netcdf4',mode='w')

rsl.close()



## Retrieve data from NOAA CO-OPS API


In [ ]:
import json
url = "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations.json"

response = requests.get(url)

# get a list of all the stations
stations = json.loads(response.text)

#from list of stations, get NOAA stations within the bounding box of UHSLC gauges
hawaii_stations = [station for station in stations['stations'] if station['lat'] >= rsl['lat'].min()-2 and station['lat'] <= rsl['lat'].max()+2 and station['lng'] >= -(360-rsl['lon'].min())-2 and station['lng'] <= -(360-rsl['lon'].max())+2]

#make a dictionary of all stations in hawaii with station name: station id
stationdictNOAA  = {station['name']: station['id'] for station in hawaii_stations}
stationdictNOAA 

# remove Pearl Harbor gauge  - tidesandcurrents lists the station that started in 2023, and we're not here to combine data.
stationdictNOAA.pop('Pearl Harbor', None)

stationdictNOAA


In [ ]:
import requests
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_data_chunk(stationID, start_date, end_date):
    # Format dates for the URL
    begin_date_str = start_date.strftime('%Y%m%d %H:%M')
    end_date_str = end_date.strftime('%Y%m%d %H:%M')
    product = 'hourly_height'
    
    # Create the URL
    server='https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?'
    url = f'{server}begin_date={begin_date_str}&end_date={end_date_str}&station={stationID}&datum=STND&product={product}&units=metric&time_zone=gmt&format=json'
    
    # Request data from NOAA API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        try:
            data = response.json()
            if 'data' in data:
                return data['data']
            else:
                # print(f"No 'data' key in response for {begin_date_str} to {end_date_str}: {data}")
                return []
        except ValueError as e:
            print(f"JSON decoding failed for {begin_date_str} to {end_date_str}: {e}")
            return []
    else:
        # print(f"Failed to fetch data for {begin_date_str} to {end_date_str}, status code: {response.status_code}")
        return []

def fetch_noaa_water_level_parallel(stationID, start_date, end_date):
    # Convert dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y%m%d %H:%M')
    end_date = datetime.strptime(end_date, '%Y%m%d %H:%M')
    
    # List to hold all data
    all_data = []
    
    # Generate date ranges in 31-day increments
    date_ranges = []
    current_start_date = start_date
    while current_start_date < end_date:
        current_end_date = current_start_date + timedelta(days=31)
        if current_end_date > end_date:
            current_end_date = end_date
        date_ranges.append((current_start_date, current_end_date))
        current_start_date = current_end_date + timedelta(seconds=1)
    
    total_ranges = len(date_ranges)
    
    # Fetch data in parallel
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_date_range = {executor.submit(fetch_data_chunk, stationID, start, end): (start, end) for start, end in date_ranges}
        completed_ranges = 0
        for future in as_completed(future_to_date_range):
            try:
                data_chunk = future.result()
                if data_chunk:
                    all_data.extend(data_chunk)
            except Exception as e:
                start, end = future_to_date_range[future]
                # print(f"Error fetching data for {start} to {end}: {e}")  #uncomment if you want to track progress
            
            completed_ranges += 1
            # print(f"Progress: {completed_ranges}/{total_ranges} chunks completed.") #uncomment if you want to track progress
    
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    if not df.empty:
        df['t'] = pd.to_datetime(df['t'])
        
        # Clean data: remove rows with empty 'v' values
        df = df[df['v'].str.strip() != '']
        
        # Convert 'v' to float
        df['v'] = df['v'].astype(float)
        
        # Remove duplicates
        df = df.drop_duplicates(subset='t')
        
        # Sort by time
        df = df.sort_values(by='t')
        
        # Create time series
        sea_level_ts = pd.Series(df['v'].values, index=df['t'])
        
        # Resample to hourly data
        sea_level_ts.index = sea_level_ts.index.round('H')
        sea_level_ts = sea_level_ts.resample('h').mean()
        
        # Ensure unique timestamps for xarray
        sea_level_ts = sea_level_ts[~sea_level_ts.index.duplicated(keep='first')]
        
        # Create xarray dataset with the correct dimension
        ds = xr.Dataset({'sea_level': ('t', sea_level_ts.values)}, coords={'t': sea_level_ts.index})

        # rename t to time
        ds = ds.rename({'t': 'time'})
        
        
        return ds
    else:
        print("No data fetched.")
        return xr.Dataset()


In [ ]:

start_date = '19050101 00:00'
end_date = '20241231 00:00'

for station_name, station_id in stationdictNOAA.items():
    # check if we already have the data
    if os.path.exists(data_dir / f'tide_gauge_nc/noaa_{station_id}.nc'):
        #check last date in nc file
        dsOLD = xr.open_dataset(data_dir / f'tide_gauge_nc/noaa_{station_id}.nc')
        last_date = dsOLD['time'].values[-1]
        dsOLD.close()
        # if last date is after end_date, skip
        if pd.to_datetime(last_date) >= datetime.strptime(end_date, '%Y%m%d %H:%M'):
            print(f"Data for {station_name} already exists and is up to date, skipping")
            continue
        
        if pd.to_datetime(last_date) < datetime.strptime(end_date, '%Y%m%d %H:%M'):
            start_date = pd.to_datetime(last_date) + timedelta(hours=1)
            start_date = start_date.strftime('%Y%m%d %H:%M')
            print(f"Data for {station_name} already exists, will append from {start_date}")
    else:
        print(f"Fetching data for {station_name}")

    ds = fetch_noaa_water_level_parallel(station_id, start_date, end_date)
    ds['station_id'] = station_id
    ds['station_name'] = station_name
    ds['station_country'] = 'USA'
    ds['lat'] = float([station['lat'] for station in hawaii_stations if station['id'] == station_id][0])
    ds['lon'] = float([station['lng'] for station in hawaii_stations if station['id'] == station_id][0])

    # if the file already exists, append the new data
    if os.path.exists(data_dir / f'tide_gauge_nc/noaa_{station_id}.nc'):
        with xr.open_dataset(data_dir / f'tide_gauge_nc/noaa_{station_id}.nc') as dsOLD:
            dsNEW = xr.concat([dsOLD, ds], dim='time')
        dsNEW = dsNEW.drop_duplicates(dim='time')
        dsNEW.to_netcdf(data_dir / f'tide_gauge_nc/noaa_{station_id}.nc')
    else:
        ds.to_netcdf(data_dir /  f'tide_gauge_nc/noaa_{station_id}.nc')

As before, we'll combine them into one dataset.

In [ ]:
##COMMENTED OUT IF DATA HAS ALREADY BEEN COMBINED< TO FIX LATER
# Load the data

      
# Get a list of all noaa*.nc files in the data directory
files = glob.glob(os.path.join(data_dir,'tide_gauge_nc','noaa*.nc'))

# Open the datasets
datasets = [xr.open_dataset(file) for file in files]
# add the station_id as a coordinate
for ds in datasets:
    # ds = remove_time_dim(ds)
    ds.coords['station_id'] = ds['station_id']
rslNOAA = xr.concat(datasets, dim='station_id',join='outer')

rslNOAA['sea_level'].attrs['units'] = 'm'
rslNOAA['sea_level'].attrs['long_name'] = 'Sea level, relative to station datum'


In [ ]:
rslNOAA

Let's add some more metadata to this dataset, including MSL and MHHW datums for each gauge.

In [ ]:
MHHW = np.zeros(len(rslNOAA['station_id']))
MSL = np.zeros(len(rslNOAA['station_id']))
MLLW = np.zeros(len(rslNOAA['station_id']))

for i in range(len(rslNOAA['station_id'])):
    url = 'https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/'+ str(rslNOAA['station_id'][i].values) +'/datums.json?units=metric'
    
    response = requests.get(url)
    datums = json.loads(response.text)
    
    def extract_datum_value(data, datum_name):
        # Iterate through the datums list
        for datum in data.get('datums', []):
            # Check if the name matches the desired datum name
            if datum.get('name') == datum_name:
                # Return the value if found
                return datum.get('value')
        # Return None if the datum name is not found
        return None
    
    #extract the MHHW and MSL datums
    MHHW[i] = extract_datum_value(datums, 'MHHW')
    MSL[i] = extract_datum_value(datums, 'MSL')
    MLLW[i] = extract_datum_value(datums, 'MLLW')




In [ ]:
rslNOAA['MHHW'] = xr.DataArray(MHHW, dims='station_id', coords={'station_id': rslNOAA['station_id']})
rslNOAA['MHHW'].attrs['units'] = 'm'
rslNOAA['MHHW'].attrs['long_name'] = 'Mean Higher High Water, rel. to station datum'

rslNOAA['MSL'] = xr.DataArray(MSL, dims='station_id', coords={'station_id': rslNOAA['station_id']})
rslNOAA['MSL'].attrs['units'] = 'm'
rslNOAA['MSL'].attrs['long_name'] = 'Mean Sea Level, rel. to station datum'

rslNOAA['MLLW'] = xr.DataArray(MLLW, dims='station_id', coords={'station_id': rslNOAA['station_id']})
rslNOAA['MLLW'].attrs['units'] = 'm'
rslNOAA['MLLW'].attrs['long_name'] = 'Mean Lower Low Water, rel. to station datum'


Change the longitude to match UHSLC 360-degree convention.

In [ ]:
rslNOAA['lon'] = rslNOAA['lon'] + 360
rslNOAA

In [ ]:
#save rsl to the data directory
#explicitly assign U50 to the station_name
rslNOAA['station_name'] = xr.DataArray(
    np.array(rslNOAA['station_name'].values, dtype='U50'),
    dims=('station_id',)
)

rslNOAA.to_netcdf(data_dir / 'rsl_hawaii_noaa.nc')

## Retrieve altimetry data 
We are using the global ocean gridded L4 [Sea Surface Heights and Derived Variables](https://doi.org/10.48670/moi-00148) from Copernicus. 

To download a subset of the global altimetry data, uncomment out the call to get_CMEMS_data and run it in this notebook. To read more about how to download the data from the Copernicus Marine Toolbox (new as of December 2023), visit https://help.marine.copernicus.eu/en/articles/7949409-copernicus-marine-toolbox-introduction. 

````{margin}
```{note}
You will need a username and password to access the CMEMS (Copernicus Marine Service) data if this is the first time running the client. To register for data access (free), visit https://data.marine.copernicus.eu/register.  
```
````

```{admonition} Large data download!
:class: warning
Getting errors on the code block below? Remember to uncomment "get_CMEMS_data()" to download. Note that if you change nothing in the function, it will download ~600 MB of data, which may take a long time!! You will only need to do this once. The dataset will be stored in the data directory you specify (which should be the same data directory we defined above).
```

In [ ]:
# get the min and max lat and lon of rsl for altimetry data retrieval
minlat = float(rslNOAA.lat.min().values)
maxlat = float(rslNOAA.lat.max().values)
minlon = float(rslNOAA.lon.min().values)
maxlon = float(rslNOAA.lon.max().values)

In [ ]:
def get_CMEMS_data(minlat, maxlat, minlon, maxlon, data_dir=data_dir):
        
    #maxlat = 15
    #minlat = 0
    #minlon = 125
    #maxlon = 140
    maxlat = maxlat+10
    minlat = minlat-10
    maxlon = maxlon+10
    minlon  = minlon-10
    
    start_date_str = "1993-01-01T00:00:00"
    end_date_str = "2025-04-30T23:59:59"
    data_dir = data_dir
    
    """
    Retrieves Copernicus Marine data for a specified region and time period.
    
    Args:
        minlon (float): Minimum longitude of the region.
        maxlon (float): Maximum longitude of the region.
        minlat (float): Minimum latitude of the region.
        maxlat (float): Maximum latitude of the region.
        start_date_str (str): Start date of the data in ISO 8601 format.
        end_date_str (str): End date of the data in ISO 8601 format.
        data_dir (str): Directory to save the retrieved data.
    
    Returns:
        str: The filename of the retrieved data.
    """
    copernicusmarine.subset(
        dataset_id="cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.125deg_P1D",
        variables=["sla"],
        minimum_longitude=minlon,
        maximum_longitude=maxlon,
        minimum_latitude=minlat,
        maximum_latitude=maxlat,
        start_datetime=start_date_str,
        end_datetime=end_date_str,
        output_directory=data_dir,
        output_filename="cmems_L4_SSH_0.125deg_" + start_date_str[0:4] + "_" + end_date_str[0:4] + "hawaii.nc"
    )
fname_cmems = 'cmems_L4_SSH_0.125deg_1993_2025hawaii.nc'

In [ ]:
import copernicusmarine

# check if the file exists, if not, download it
if not os.path.exists(data_dir / fname_cmems):
    print('You will need to download the CMEMS data in a separate script')
    get_CMEMS_data(minlat, maxlat, minlon, maxlon, data_dir) #<<--- COMMENT OUT TO AVOID ACCIDENTAL DATA DOWNLOADS.
else:
    print('CMEMS data already downloaded, good to go!')

Open up the CMEMS data and take a look. We will want to make an ASL dataset similar in structure to the RSL data so that we can easily compare the two.

In [ ]:
# open the CMEMS data
fname_cmems = 'cmems_L4_SSH_0.125deg_1993_2025hawaii.nc'
ds = xr.open_dataset(data_dir / fname_cmems)

ds

In [ ]:
# use NOAA data
rsl = xr.open_dataset(data_dir / 'rsl_hawaii_noaa.nc')

In [ ]:
rsl

In [ ]:
# Extract data for the nearest point to the tide gauge location that has data
sla = []
for lat, lon in zip(rsl['lat'].values, rsl['lon'].values):
    sla.append(ds['sla'].sel(
        longitude=lon, latitude=lat, method='nearest'
    ))

    #if the data is null, nan average over the nearest 4 points
    tol = 0.25
    if sla[-1].isnull().all():
        sla[-1] = ds['sla'].sel(
            longitude=slice(lon-360-tol, lon-360+tol), 
            latitude=slice(lat-tol, lat+tol)
        ).mean(dim=['latitude', 'longitude'])
        sla[-1]['latitude'] = np.mean(lat)
        sla[-1]['longitude'] = np.mean(lon)

sla = xr.concat(sla, dim='station_id')

# make sla a dataset with variables from rsl
sla = sla.to_dataset(name='sla')
sla['station_name'] = rsl['station_name']

# Creating lat_str and lon_str arrays with 'station_id' as their dimension
lat_str = [f'{np.abs(lat):.3f}\u00B0{"N" if lat > 0 else "S"}' for lat in sla.latitude.values]
lon_str = [f'{np.abs(lon):.3f}\u00B0{"E" if lon > 0 else "W"}' for lon in sla.longitude.values]  

# Convert lists to DataArrays with 'station_id' as their dimension
lat_str_da = xr.DataArray(lat_str, dims=['station_id'], coords={'station_id': rsl['station_id']})
lon_str_da = xr.DataArray(lon_str, dims=['station_id'], coords={'station_id': rsl['station_id']})

# Assign these DataArrays to the sla dataset
sla['lat_str'] = lat_str_da
sla['lon_str'] = lon_str_da

# add original data source to attributes
sla.attrs['original_data_source'] = 'CMEMS L4 SSH 0.125deg'
sla.attrs['title'] = ds.attrs['title']
sla.attrs['source_file'] = str(data_dir / fname_cmems)

# ensure latitude and longitude are coordinates associated with a location
sla = sla.set_coords(['latitude', 'longitude'])



sla

Careful here! The mean SLA for the 1993-2012 reference period is NOT zero, it is near 2.5cm, due to ensuring that the 1993 mean sea level is zero. We'll use this later when making our tide gauge comparison.

In [ ]:
#save sla to the data directory
sla.to_netcdf(data_dir / 'asl_hawaii.nc')

## Processing
### Process the tide gauge data to match CMEMS
 Now we'll convert tide gauge data into a daily record for the POR in units of meters to match the CMEMS data. 
 
 The next code block:
 - extracts tide gauge data for the period 1993-2022
 - converts it to meters
 - removes any NaN values
 - resamples the data to daily mean
 - and normalizes it relative to the 1993-2012 epoch. 
 

The resulting data is stored in the variable 'rsl_daily' with units in meters.

In [ ]:
# Next, let's establish a period of record from 1993-2022.
# establish the time period of interest
start_date = dt.datetime(1993,1,1)
end_date = dt.datetime(2024,12,31)
#
# also save them as strings, for plotting
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')



In [ ]:
# Extract the data for the period of record (POR)
tide_gauge_data_POR = rsl['sea_level'].sel(time=slice(start_date, end_date))


## Get daily tide gauge data to match altimetry record
To make a fair comparison with the CMEMS satellite data, we first need to remove the tides from our tide-gauge records (isn't it ironic). The satellite product (a daily value) already has the regular daily and twice-daily tidal signals taken out, so we do the same for the gauge data. To do this, we apply a Godin filter — a well-established method in oceanography that smooths hourly sea level records with three succesive running moving averages (24, 24, and 25 hours long). This process effectively strips away the repeating tidal cycles and leaves behind the slower, non-tidal changes in sea level that we want to study. 


```{margin} What's daily data, anyway?
The University of Hawai‘i Sea Level Center (UHSLC) daily product uses a 119-point convolution filter — a weighted ~5-day low-pass window with maximum weight at the center (noon) and tapering at the edges. This filter acts very much like the Godin filter in that it removes tidal energy at 1 and 2 cycles per day, but is somewhat smoother, further damping signals with periods shorter than a few days. Both filters are suitable for tide-gauge/altimetry comparisons for general trend calculations, although direct removal of predicted tidal signals should in theory align more directly with altimetry products. The UHSLC approach, described by {cite:t}`kilonsky_pursuit_1991`, first performs a harmonic tidal analysis (with UTide) and removes all constituents with periods shorter than 30 h. The resulting residual hourly series is then averaged with the noon-centered 119-hour filter, producing a daily sea level time series.   
```

In [ ]:
def godin_xr(da: xr.DataArray, allow_missing: float = 0.0) -> xr.DataArray:
    """
    Godin filter (24h → 24h → 25h centered moving averages) along 'time'.
    - da: DataArray with a 'time' dimension (hourly, sorted).
    - allow_missing: fraction of a window allowed to be NaN before output is NaN.
        0.0 = strict Godin (full window required)
        0.25 = allow up to 25% missing within each window
    """
    assert "time" in da.dims, "Input must have a 'time' dimension."

    def ma(x, n):
        # min samples needed in the n-hour window
        min_periods = int(np.ceil((1.0 - allow_missing) * n))
        return x.rolling(time=n, center=True, min_periods=min_periods).mean()

    y = ma(da, 24)
    y = ma(y, 24)
    y = ma(y, 25)
    return y

In [ ]:
sea_level_detided = godin_xr(tide_gauge_data_POR)

Now we have filtered hourly data, so let's resample that to noon values. We'll do this to align with the altimetry data (and UHSLC's 119-point noon-centered filter).

In [ ]:
# pick one sample per day: closest to 12:00
t = pd.DatetimeIndex(sea_level_detided.time.values)
frac_hours = t.hour + t.minute/60 + t.second/3600
dist = np.abs(frac_hours - 12.0)
noon_idx = (
    pd.Series(dist, index=np.arange(len(t)))
    .groupby(t.normalize())
    .idxmin()
    .to_numpy()
)

daily = sea_level_detided.isel(time=noon_idx).rename({'time': 'time_noon'})
daily = daily.assign_coords(time_noon=sea_level_detided.time.isel(time=noon_idx).values)

# drop any NaNs from Godin edge effects
daily = daily.dropna(dim='time_noon', how='all')

# rename time_noon to time
daily = daily.rename({'time_noon': 'time'})

In [ ]:
# compare daily data from different sources, let's do Honolulu, which is 57 for UHSLC and 1612340 for NOAA
url = "https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/" 
uhslc_id = '057'
fname = f'd{uhslc_id}.nc' # h for hourly, d for daily
path = os.path.join(data_dir, 'tide_gauge_nc',fname)

if not os.path.exists(path):
    urlretrieve(os.path.join(url, fname), path) 
    print(f'Downloading {fname} from {url} to {path}')

# open the UHSLC data
rsl_uhslc_daily = xr.open_dataset(path)
# change record_id to station_id
rsl_uhslc_daily = rsl_uhslc_daily.rename({'record_id': 'station_id'})


In [ ]:
# compare the two datasets
rsl_uhslc_daily = rsl_uhslc_daily.sel(time=slice(start_date + pd.DateOffset(days=1), end_date - pd.DateOffset(days=1)))
rsl_uhslc_daily['sea_level_m'] = rsl_uhslc_daily['sea_level']/1000   # convert from mm to m


# truncate both datasets to end in 2019-12-31, start on 1993-01-02
daily = daily.sel(time=slice('1993-01-02', '2019-12-31'))
rsl_uhslc_daily = rsl_uhslc_daily.sel(time=slice('1993-01-02', '2019-12-31'))


# make a figure with 3 rows and 1 column
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
fig,axs = plt.subplots(3,1,figsize=(10,15))
# first plot the full time series
axs[0].plot(daily.sel(station_id='1612340').time, daily.sel(station_id='1612340'), label='Godin', color='blue')
axs[0].plot(rsl_uhslc_daily.sel(station_id=570).time, rsl_uhslc_daily.sel(station_id=570).sea_level_m, label='UHSLC', color='orange')
axs[0].set_ylabel('Sea Level (m)')
axs[0].legend()
axs[0].set_title('Full Time Series')
# axs[0].set_xlim(pd.to_datetime('2023-01-01'), pd.to_datetime('2023-12-31'))

# second plot the difference
axs[1].plot(daily.sel(station_id='1612340').time, 1000*(daily.sel(station_id='1612340') - rsl_uhslc_daily.sel(station_id=570).sea_level_m), label='Difference', color='green')
axs[1].set_ylabel('Difference (mm)')
axs[1].legend()
axs[1].set_title('Difference Between Godin and UHSLC')

# third plot the scatter plot
axs[2].scatter(rsl_uhslc_daily.sel(station_id=570).sea_level_m, daily.sel(station_id='1612340'), color='purple', alpha=0.5)
axs[2].set_xlabel('UHSLC Daily (m)')
axs[2].set_ylabel('Godin Daily (m)')
axs[2].set_title('Scatter Plot')


# # add bias and rmsd to the plot
bias = (daily.sel(station_id='1612340') - rsl_uhslc_daily.sel(station_id=570).sea_level_m).mean().values
rmsd = np.sqrt(((daily.sel(station_id='1612340') - rsl_uhslc_daily.sel(station_id=570).sea_level_m)**2).mean()).values
plt.title(f'Comparison of Godin and UHSLC Daily Sea Level at Honolulu\nBias: {bias*1000:.2f} mm, RMSD: {rmsd*1000:.2f} mm')

# # set date to be all of 2017
# plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2017-12-31'))
# plt.ylim(1.4,1.9)

# axs[2].scatter(rsl_uhslc_daily.sel(station_id=570).sea_level_m, daily.sel(station_id='1612340'), color='green', alpha=0.5)
# # plot 1:1 line
# lims = [min(rsl_uhslc_daily.sel(station_id=570).sea_level_m.min().values, daily.sel(station_id='1612340').min().values), 
#         max(rsl_uhslc_daily.sel(station_id=570).sea_level_m.max().values, daily.sel(station_id='1612340').max().values)]
# axs[2].plot(lims, lims, 'k--', alpha=0.75, zorder=0)
# axs[2].set_xlim(lims)
# axs[2].set_ylim(lims)
# axs[2].set_xlabel('UHSLC Daily (m)')
# axs[2].set_ylabel('Godin Daily (m)')
# axs[2].set_title('Scatter Plot')
# set aspect ratio to 1:1
axs[2].set_aspect('equal', 'box')



### Normalize the tide gauge data relative to the 1993-2012 epoch

In [ ]:

epoch_start, epoch_end = start_date, '2012-12-31'
epoch_daily_avg = daily.sel(time=slice(epoch_start, epoch_end))
epoch_daily_mean = epoch_daily_avg.mean(dim='time')



In [ ]:
epoch_daily_mean_cmems = sla.sel(time=slice(epoch_start, epoch_end)).mean(dim='time')
epoch_daily_mean_cmems

In [ ]:
epoch_daily_mean

In [ ]:
# Display station names in a cleaner format
print("Station Names:")
for i, name in enumerate(rsl['station_name'].values):
    station_id = rsl['station_id'].values[i]
    print(f"{i}: {name} (Station ID: {station_id})")

In [ ]:
start_date

In [ ]:
# Subtract the epoch daily mean from the tide gauge daily average
rsl_daily = daily - epoch_daily_mean

# add in the CMEMS mean for the epoch to remove bias from 1993 start.
rsl_daily = rsl_daily + epoch_daily_mean_cmems['sla']

# Set the attributes of the rsl_daily data
rsl_daily.attrs = tide_gauge_data_POR.attrs
rsl_daily.attrs['units'] = 'm'

# add lat and lon to the dataset
rsl_daily['lat'] = rsl['lat']
rsl_daily['lon'] = rsl['lon']

# add the station name and country
rsl_daily['station_name'] = rsl['station_name']

# change the variable name to sea level anomaly
rsl_daily.name = 'rsl_anomaly'

# change long name of the variable to sea level anomaly
rsl_daily.attrs['long_name'] = 'Sea Level Anomaly'
rsl_daily.attrs['epoch'] = '1993-2012'

rsl_daily

In [ ]:
g = rsl_daily.plot(x='time', col='station_id', col_wrap=1, sharey=False, sharex=True, figsize=(15, 10))

# Use g.axs to iterate over the axes in the FacetGrid
for ax, id in zip(g.axs.flat, rsl_daily.station_id):
    # Accessing the station_name coordinate for the current station_id directly
    station_name = rsl_daily.station_name.sel(station_id=id).item()
    ax.set_title(station_name)

plt.show()


In [ ]:
stationdictNOAA

In [ ]:
# Set tide gauge daily average relative to MHHW
rsl_daily_mhhw = daily - rsl['MHHW']

# Set the attributes of the rsl_daily data
rsl_daily_mhhw.attrs = tide_gauge_data_POR.attrs
rsl_daily_mhhw.attrs['long_name'] = 'water level above MHHW'
rsl_daily_mhhw.attrs['units'] = 'm'
# add lat and lon to the dataset
rsl_daily_mhhw['lat'] = rsl['lat']
rsl_daily_mhhw['lon'] = rsl['lon']
# add the station name and country
rsl_daily_mhhw['station_name'] = rsl['station_name']

rsl_daily_mhhw.name = 'rsl_mhhw'
# save rsl_daily to the data directory
# rsl_daily_mhhw.to_netcdf(data_dir / 'rsl_daily_hawaii_mhhw.nc')


#combine the two datasets
rsl_daily_combined = xr.merge([rsl_daily, rsl_daily_mhhw])

rsl_daily_combined['storm_time'] = rsl_daily_combined['time']
rsl_daily_combined['storm_time'] = xr.DataArray(
    pd.to_datetime(rsl_daily_combined['storm_time'].values).to_series().apply(
        lambda x: x if x.month >= 5 else x - pd.DateOffset(years=1)
    ),
    dims=rsl_daily_combined['storm_time'].dims,
    coords=rsl_daily_combined['storm_time'].coords
)

#make storm time a coordinate
rsl_daily_combined = rsl_daily_combined.assign_coords(storm_time = rsl_daily_combined.storm_time, compat='no_conflicts')

# save rsl_daily_combined to the data directory
rsl_daily_combined.to_netcdf(data_dir / 'rsl_daily_hawaii.nc')

In [ ]:
rsl_daily_combined

In [ ]:
#what's the mean sla_tg anomaly from 1993-2012
sla_mean = sla.sel(time=slice('1993-01-01', '2012-12-31')).mean(dim='time')
sla_mean


In [ ]:
# rsl_daily_combined['storm_time'] = rsl_daily_combined['time']
# rsl_daily_combined['storm_time'] = xr.DataArray(
#     pd.to_datetime(rsl_daily_combined['storm_time'].values).to_series().apply(
#         lambda x: x if x.month >= 5 else x - pd.DateOffset(years=1)
#     ),
#     dims=rsl_daily_combined['storm_time'].dims,
#     coords=rsl_daily_combined['storm_time'].coords
# )

# #make storm time a coordinate
# rsl_daily_combined = rsl_daily_combined.assign_coords(storm_time = rsl_daily_combined.storm_time)

# # save rsl_daily_combined to the data directory
# rsl_daily_combined.to_netcdf(data_dir / 'rsl_daily_hawaii.nc')

# rsl_daily_combined


## Retrieve Climate Indices

In [ ]:
def read_psl_url(url):
    response = requests.get(url)
    data = response.text
    # Split the data into lines
    lines = data.strip().split('\n')
    # Collect data lines until we encounter -9999
    data_lines = []
    for line in lines[1:]:  # Skip the first line if it's a header
        stripped_line = line.strip()
        if stripped_line == '':
            continue  # Skip empty lines
        # Detect possible footer lines (either starting with -99.9, -9999, or a textual footer)
        if stripped_line.startswith('-99.9') or stripped_line.startswith('-99.99') or stripped_line.startswith('-9999') or any(char.isalpha() for char in stripped_line):
            break  # Stop processing when footer is encountered
        # Add valid data lines to the list
        data_lines.append(line)
    # Combine data lines into a single string
    data_str = '\n'.join(data_lines)
    data_io = io.StringIO(data_str)
    return data_io

In [ ]:
def read_psl_format(csvfile, varName):
    df = pd.read_csv(csvfile, sep=r'\s+', skiprows=1, skipfooter=0, header=None, engine='python')
    CI = df.melt(id_vars=0, var_name='Month', value_name=varName)
    #Replace placeholder values with NaN
    CI.replace([-9.99, -9999, -9999.000, -999.99,-99.90,-99.99], np.nan, inplace=True)
    # Drop rows with NaN values
    CI = CI.dropna(subset=[varName])
    CI['time'] = pd.to_datetime({'year': CI[0], 'month': CI['Month'], 'day': 1})
    # Sort by 'time'
    CI = CI.sort_values('time').reset_index(drop=True)
    CI = CI[['time', varName]]

    return CI

In [ ]:
def read_ncei_format(csvfile, varName):
    df = pd.read_csv(csvfile, sep=r'\s+', skiprows=0, skipfooter = 0,header=1, engine='python')
    ClimateIndex = df.melt(id_vars='Year', var_name='Month', value_name=varName)
    ClimateIndex['Month'] = ClimateIndex['Month'].map(month_mapping)
    # Replace placeholder values with NaN
    ClimateIndex.replace(99.99, np.nan, inplace=True)
    # Drop rows with NaN values
    ClimateIndex = ClimateIndex.dropna(subset=[varName])
    ClimateIndex['time'] = pd.to_datetime({'year': ClimateIndex['Year'], 'month': ClimateIndex['Month'], 'day': 1})
    # Sort by 'time'
    ClimateIndex = ClimateIndex.sort_values('time').reset_index(drop=True)
    ClimateIndex = ClimateIndex[['time', varName]]
    return ClimateIndex

In [ ]:
month_mapping = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
    'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
    'Sep': 9, 'Oct': 10, 'Nov':11, 'Dec':12
}

In [ ]:
# Function to convert fraction of the year to datetime
def convert_fraction_to_date(date_float):
    # Extract year and fraction
    year = int(date_float)
    fraction = date_float - year
    
    # Calculate the total number of days in the year (consider leap years)
    days_in_year = 366 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 365
    
    # Calculate the day of the year based on the fraction
    day_of_year = int(np.floor(fraction * days_in_year)) + 1
    
    # Create a datetime object for January 1st of the given year
    start_date = pd.to_datetime(f"{year}-01-01")
    
    # Add the day offset to get the actual date
    return start_date + pd.to_timedelta(day_of_year - 1, unit='D')

## Pacific Meridional Mode

In [ ]:
def retrieve_pmm(yearStart='1950', yearEnd='2024', data_dir=data_dir):
    plottingUrlBase = 'https://psl.noaa.gov/cgi-bin/data/climateindices/corr.pl?'
    plotSpecs1 = 'tstype1=45&custname1=&custtitle1=&tstype2=0&custname2=&custtitle2=&'
    timeRange = 'year1=' + yearStart + '&year2=' + yearEnd + '&'
    plotSpecs2 = 'itypea=0&y1=&y2=&plotstyle=0&length=&lag=&iall=0&iseas=0&mon1=0&mon2=11&anom=0&climo1_yr1=&climo1_yr2=&climo2_yr1=&climo2_yr2=&Submit=Calculate+Results'

    plottingUrl = plottingUrlBase + plotSpecs1 + timeRange + plotSpecs2

    # Access the plotting URL and search for the "Data file" link
    response = requests.get(plottingUrl)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch plotting URL: {response.status_code}")

    # Use a regular expression to find the "Data file" link
    match = re.search(r'href=([\'"]?)(/tmp/gcos_wgsp/[^\'"\s>]+)\1>Data file<', response.text)
    if not match:
        raise Exception("Data file link not found in the response.")
    
    data_url = match.group(2)

    # Download the data
    response = requests.get('https://psl.noaa.gov/' + data_url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data file: {response.status_code}")

    data = response.text

    # Read the data into a DataFrame
    data_io = io.StringIO(data)
    pmm = pd.read_csv(data_io, sep=r'\s+', header=None, names=["time", "PMM"])

    # Remove rows with missing values
    pmm = pmm.dropna()

    # Apply the conversion function to the 'Date' column
    pmm['time'] = pmm['time'].apply(convert_fraction_to_date)

    # Put this time as closest to the first day of the month
    pmm['time'] = pmm['time'] + pd.DateOffset(days=15)
    pmm['time'] = pmm['time'].dt.to_period('M').dt.to_timestamp()

    # Save to the data directory without the index
    pmm.to_csv(data_dir / 'climate_indices' / 'pmm.csv', index=False)
    print('PMM data retrieved and saved to data directory')
    return pmm

## PDO: Pacific Decadal Oscillation

Monthly SST anomalies in the North Pacific Ocean. 

In [ ]:
def retrieve_pdo_ncei(data_dir=data_dir):
    url = 'https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.pdo.dat'
    response = requests.get(url)
    data = response.text
    data_io = io.StringIO(data)
    PDO = read_psl_format(data_io, 'PDO')
    PDO.to_csv(data_dir / 'pdo.csv', index=False)
    print('PDO data retrieved and saved to data directory')

    return PDO


## BEST: Bivariate EnSo Timeseries 

This climate index combines the atmospheric component of ENSO (SOI) with the oceanic component (Nino 3.4 SST). 

For more info, see: https://psl.noaa.gov/people/cathy.smith/best/details.html
Smith, C.A. and P. Sardeshmukh, 2000, The Effect of ENSO on the Intraseasonal Variance of Surface Temperature in Winter., International J. of Climatology, 20 1543-1557.

In [ ]:
def retrieve_climateIndex(ClimateIndex, data_dir=data_dir):
    PSLurlBase = 'https://psl.noaa.gov/data/'

    # Define the URL based on the ClimateIndex
    if ClimateIndex in ['TNA', 'ONI','PNA']:
        url = PSLurlBase + 'correlation/' + ClimateIndex.lower() + '.data'
    
    elif ClimateIndex in ['AAO']:
        url = PSLurlBase + 'timeseries/month/data/' + ClimateIndex.lower() + '.data'

    elif ClimateIndex == 'AO':
        url = PSLurlBase + 'timeseries/month/data/' + ClimateIndex.lower() + '.long.data'

    elif ClimateIndex == 'DMI':
        url = 'https://psl.noaa.gov/gcos_wgsp/Timeseries/Data/dmieast.had.long.data'

    elif ClimateIndex == 'BEST':
        url = PSLurlBase + 'correlation/censo.data'
    
    elif ClimateIndex == 'PDO':
        url = 'https://psl.noaa.gov/pdo/data/pdo.timeseries.sstens.data'

    else:
        raise ValueError(f"ClimateIndex '{ClimateIndex}' is not recognized")
    
    print(f"Retrieving data from: {url}")

    # Fetch and format the data
    data_io = read_psl_url(url)
    climate_data = read_psl_format(data_io, ClimateIndex)

    # Save the data to a CSV file
    output_file = data_dir / 'climate_indices' / (ClimateIndex.lower() + '.csv')
    climate_data.to_csv(output_file, index=False)
    print(f"{ClimateIndex} data retrieved and saved to {output_file}")

    return climate_data

In [ ]:
ONI = retrieve_climateIndex('ONI')
TNA = retrieve_climateIndex('TNA')
PNA = retrieve_climateIndex('PNA')
# AAO = retrieve_climateIndex('AAO')
AO = retrieve_climateIndex('AO')
# DMI = retrieve_climateIndex('DMI')
BEST = retrieve_climateIndex('BEST')

PDO = retrieve_climateIndex('PDO')

import re
PMM = retrieve_pmm()

In [ ]:
# List of all DataFrames and their corresponding names
CIs = {
    'BEST': BEST,
    # 'DMI': DMI,
    'AO': AO,
    # 'AAO': AAO,
    'PNA': PNA,
    'TNA': TNA,
    'ONI': ONI,
    'PDO': PDO,
    'PMM': PMM
}

# Start with the first DataFrame (BEST in this case)
merged_CI = CIs['BEST']

# Loop through the rest of the DataFrames and merge them on 'time'
for name, df in CIs.items():
    if name != 'BEST':
        merged_CI = pd.merge(merged_CI, df, on='time', how='outer')



In [ ]:
# truncate to 1990 through 2024
merged_CI = merged_CI[(merged_CI['time'] >= '1990-01-01') & (merged_CI['time'] <= '2024-12-31')]

#replace placeholder values with NaN
merged_CI.replace([-999], np.nan, inplace=True)

merged_CI.to_csv(data_dir / 'climate_indices' / 'climate_indices.csv', index=False)


In [ ]:
# get standard deviation of the climate indices
merged_CI = pd.read_csv(data_dir / 'climate_indices' / 'climate_indices.csv')

# Calculate the standard deviation of each column except 'time'
std_dev = merged_CI.drop(columns='time').std()

# calculate the mean of each column except 'time'
mean = merged_CI.drop(columns='time').mean()

In [ ]:
mean

In [ ]:
# normalize the climate indices such that they have a standard deviation of 1
merged_CI_norm = merged_CI.copy()
for col in merged_CI_norm.columns[1:]:
    # normalize the column by subtracting the mean and dividing by the standard deviation
    merged_CI_norm[col] = merged_CI_norm[col] - mean[col]
    merged_CI_norm[col] = merged_CI_norm[col] / std_dev[col]

merged_CI_norm['time'] = pd.to_datetime(merged_CI_norm['time'])

#save to the data directory
merged_CI_norm.to_csv(data_dir / 'climate_indices' / 'climate_indices_norm.csv', index=False)

    

In [ ]:
merged_CI_norm

In [ ]:
import matplotlib.dates as mdates

indices = merged_CI_norm.columns[1:]

# Adjusting the x-axis limits to range from 1993 to 2024
fig, axs = plt.subplots(len(indices), 1, figsize=(10, 12), sharex=True)

# Loop through each index and its corresponding subplot
for i, index in enumerate(indices):
    axs[i].plot(merged_CI_norm['time'], merged_CI_norm[index], label=index)
    axs[i].set_ylabel(index)  
    axs[i].set_ylim(-3, 3)  # Set y-axis limits to
    axs[i].tick_params(axis='y', labelsize=8)  # Set smaller tick size for y-axis

# Set the x-axis label only for the last subplot
axs[-1].set_xlabel('Year')

# Set x-axis major ticks to every 5 years and limit to 1993 to 2024
axs[-1].set_xlim(pd.Timestamp('1990-01-01'), pd.Timestamp('2024-01-01'))
axs[-1].xaxis.set_major_locator(mdates.YearLocator(5))
axs[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Set a common title for the figure
fig.suptitle('Normalized Climate Indices over Time', fontsize=16)

# Adjust spacing between subplots, reducing vertical gaps
plt.tight_layout(rect=[0, 0, 1, 0.96], h_pad=0.5)




---

## Citations

```{bibliography}
:style: plain
:filter: docname in docnames
```